In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy.interpolate import UnivariateSpline
import heapq
import time


timestamp = int(time.time())
root_folder = f'raw/{str(timestamp)}'
train_folder = f'{root_folder}/train'
if not os.path.exists(train_folder):
    os.makedirs(train_folder)


val_folder = f'{root_folder}/val'
if not os.path.exists(val_folder):
    os.makedirs(val_folder)


t_steps = 200
obs_avoid_dist = 3
num_train, num_val = 9000, 1000
env_dpi = 16  # figsize = (2, 2) -> 32x32 pixels


# Helper functions for A*
def heuristic(a, b):
    return np.sqrt((b[0] - a[0]) ** 2 + (b[1] - a[1]) ** 2)

def get_neighbors(node):
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0), (1, 1), (-1, -1), (1, -1), (-1, 1)]
    result = []
    for d in directions:
        neighbor = (node[0] + d[0], node[1] + d[1])
        if 0 <= neighbor[0] < 30 and 0 <= neighbor[1] < 30:
            result.append(neighbor)
    return result

def a_star(start, goal, obstacles):
    open_set = []
    heapq.heappush(open_set, (0, start))
    came_from = {}
    g_score = {start: 0}
    f_score = {start: heuristic(start, goal)}
    
    while open_set:
        current = heapq.heappop(open_set)[1]
        
        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            return path[::-1] # Return reversed path
        
        for neighbor in get_neighbors(current):
            if neighbor in obstacles:
                continue
            
            tentative_g_score = g_score[current] + heuristic(current, neighbor)
            
            if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = g_score[neighbor] + heuristic(neighbor, goal)
                heapq.heappush(open_set, (f_score[neighbor], neighbor))
    
    return None


def check_valid(x, y, x0, h0, x1, h1):
    if np.any(np.isnan(y)) or not (np.all(0 <= y) and np.all(y <= 30)):
        return False
    
    # Check if the path is valid
    for i in range(len(x)):
        if x0 - obs_avoid_dist <= x[i] <= x0 + obs_avoid_dist and 0 <= y[i] <= h0:
            return False
        if x1 - obs_avoid_dist <= x[i] <= x1 + obs_avoid_dist and 30 - h1 <= y[i] <= 30:
            return False
    return True


for train_val in range(2):
    folder = train_folder if train_val == 0 else val_folder
    num_envs = num_train if train_val == 0 else num_val
    ind = 0
    while ind < num_envs:
        # if plt is open, close it
        plt.close()
        # Step 1: Randomly pick variables
        x0, x1 = np.random.randint(2, 28, 2)
        while abs(x1 - x0) < 5:
            x1 = np.random.randint(2, 28)

        x0, x1 = np.sort([x0, x1])
        h0, h1 = np.random.randint(5, 26, 2)
        s, e = np.random.randint(0, 31, 2)

        plt.figure(figsize=(2, 2), dpi=env_dpi)
        plt.xlim(-0.5, 30.5)
        plt.ylim(0, 30)
        frame = plt.gca()
        frame.set_facecolor('black')

        plt.plot([x0, x0], [0, h0], color='red', linewidth=5)
        plt.plot([x1, x1], [30-h1, 30], color='red', linewidth=5)

        # Defining obstacles based on x0, x1, h0, and h1
        obstacles = set()
        for i in range(h0 + obs_avoid_dist):
            obstacles.add((x0, i))
        for i in range(30 - h1 - obs_avoid_dist, 30):
            obstacles.add((x1, i))

        # Finding a path
        start = (0, s)
        goal = (29, e)

        path = a_star(start, goal, obstacles)
        if path is None:
            continue
        path = np.clip(path, 0, 30)

        path_array = np.array(path)

        x = path_array[:, 0]
        y = path_array[:, 1]


        degree = 3  # Degree of the smoothing spline
        spl = UnivariateSpline(x, y, s=len(path)) #make_interp_spline(x, y, k=degree)

        # Generate new, smoother x and y values
        x_smooth = np.linspace(x.min(), x.max(), t_steps)
        y_smooth = spl(x_smooth)

        # Check if the path is valid
        is_valid = check_valid(x_smooth, y_smooth, x0, h0, x1, h1)

        if not is_valid:
            continue

        frame.axes.get_xaxis().set_visible(False)
        frame.axes.get_yaxis().set_visible(False)
        
        plt.subplots_adjust(left=0,
                        bottom=0,
                        right=1,
                        top=1)
        
        plt.savefig(os.path.join(folder, f'env_{ind}.png'))


        # show start and goal with green circles
        plt.plot(x_smooth[0], y_smooth[0], 'go', markersize=10)
        plt.plot(x_smooth[-1], y_smooth[-1], 'go', markersize=10)

        plt.plot(x_smooth, y_smooth, color='blue', linewidth=3)

        frame.axes.get_xaxis().set_visible(True)
        frame.axes.get_yaxis().set_visible(True)
        plt.xlabel('x', color='white')
        plt.ylabel('y', color='white')

        # Save the plot
        plt.savefig(os.path.join(folder, f'env_soln_{ind}.png'))

        # save traj in a .npy file
        np.save(os.path.join(folder, f'traj_{ind}.npy'), np.array([x_smooth, y_smooth]))

        # save the environment information (x0, h0, x1, h1, s, e) in a dictionary and then in a .npy file
        env_info = {'x0': x0, 'h0': h0, 'x1': x1, 'h1': h1, 's': s, 'e': e}
        np.save(os.path.join(folder, f'env_info_{ind}.npy'), env_info)

        ind += 1
        plt.close()

/home/yigit/.local/lib/python3.8/site-packages/scipy/interpolate/_fitpack2.py:280: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)
